In [11]:
from pyrocko import util, model, io, trace, moment_tensor, gmtpy,orthodrome
from pyrocko import orthodrome as od
from pyrocko.client import catalog
from pyrocko.automap import Map
import pyrocko.moment_tensor as pmt
from pyrocko.plot import mpl_color
from pyrocko.guts import load
# from seiscloud import plot as scp
# from seiscloud import cluster as scc
import numpy as np
import os
# import shutil
import matplotlib.pyplot as plt
import pygmt
import grond

In [12]:
workdir='../'
reportdir=os.path.join(workdir,'report')                                

catdir=os.path.join(workdir,'CAT')
catname=os.path.join(catdir,'catalogue_flegrei_VLP.pf')                 # CHANGE
refevents=model.load_events(catname)
mttargets = [ev for ev in refevents]

badmtsols = ['']    # exclude some events
print(f'Catalogue: {catname}')
print('All events in catalogue:', len(mttargets))
goodmttargets = [ev for ev in mttargets if ev.name not in badmtsols]
print('Good events in catalogue:', len(goodmttargets))

# Insert inversions names
inversions= ['cmt_composite_VT_VLP_']                                    # CHANGE

# Insert source prefixes 
source_prefixes = ['vt','vlp']                                             # CHANGE
par_names = ['time','north_shift','east_shift',
              'depth','magnitude',
              'rmnn','rmee','rmdd','rmne','rmnd','rmed',
              'duration','frequency',
              'strike1','dip1','rake1']

print(f'\nInversions selected: {inversions}')
print(f'Sources extracted: {source_prefixes}')

# new catalogue name
new_catalogue_name = 'catalogue_flegrei_MT_composite'

Catalogue: ../CAT/catalogue_flegrei_VLP.pf
All events in catalogue: 14
Good events in catalogue: 14

Inversions selected: ['cmt_composite_VT_VLP_']
Sources extracted: ['vt', 'vlp']


In [13]:
#events_names = []
events = {source_name: [] for source_name in source_prefixes}


for inv in inversions:
    for ev in goodmttargets:
        targetdir = os.path.join(reportdir, ev.name, inv + ev.name)
        if os.path.isdir(targetdir):
            fname = os.path.join(targetdir, 'stats.yaml')     
            stats=load(filename=fname)

            # store event name
            tmp_name=stats.problem.name[len(inv):]
            #events_names.append(tmp_name)

            # create new dict for selected event
            parameter = {source_name: [] for source_name in source_prefixes}
            for source_name in parameter:
                parameter[source_name] = {par_name: [] for par_name in par_names}

            # add perameters to dict
            for stat in stats.parameter_stats_list: 
                stat_prefix,stat_name = stat.name.split('.')
                if stat_prefix in parameter:
                    if stat_name in parameter[stat_prefix]:
                        parameter[stat_prefix][stat_name]= stat.best # take BEST results
                else:
                    print(f'Warning: parameter prefix not found:')
                    print(f'current prefix {stat_prefix} =/= selected prefixes {source_prefixes}')

            # create events objects for sub-problems
            for source_name in parameter:
                m0=pmt.magnitude_to_moment(parameter[source_name]['magnitude'])
                tmp_mt = pmt.MomentTensor(mnn=parameter[source_name]['rmnn'] * m0,
                                        mee=parameter[source_name]['rmee'] * m0,
                                        mdd=parameter[source_name]['rmdd'] * m0,
                                        mne=parameter[source_name]['rmne'] * m0,
                                        mnd=parameter[source_name]['rmnd'] * m0,
                                        med=parameter[source_name]['rmed'] * m0,
                                        moment=m0)

                if source_name == 'vlp':    # re-locate
                    nlat, nlon = od.ne_to_latlon(ev.lat, ev.lon,
                                    np.array([parameter[source_name]['north_shift']]),
                                    np.array([parameter[source_name]['east_shift']]))
                    lat, lon = nlat[0], nlon[0]
                else:
                    lat, lon = ev.lat , ev.lon

                time = ev.time + parameter[source_name]['time']
                tmp_ev=model.Event(lat=lat, 
                                   lon=lon, 
                                   time=time,
                                   name=tmp_name, 
                                   depth=parameter[source_name]['depth'], 
                                   elevation=None, 
                                   magnitude=parameter[source_name]['magnitude'], 
                                   moment_tensor=tmp_mt, 
                                   duration=parameter[source_name]['duration'], 
                                   tags=['sub-problem:', source_name,'frequency:' ,str(parameter[source_name]["frequency"]) ] )            
                # Add event to dict events
                events[source_name].append(tmp_ev)
        #else:
            #print('missing report dir "', inv, '" for event:',ev.name)

In [14]:
#events.sort(key=lambda x: x.time, reverse=False)
print(f'total sub events: {len(events)}')

# save
for source in source_prefixes:
    new_catalogue_path=os.path.join(catdir,new_catalogue_name + '_' + source + '.pf')   
    model.dump_events(events[source], new_catalogue_path)


total sub events: 2
